# Working with Verily Workbench groups

<table align="left">

  <td>
    <a href="https://github.com/DataBiosphere/terra-axon-examples/blob/main/first_hour_on_terra/working_with_groups.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/DataBiosphere/terra-axon-examples/main/first_hour_on_terra/working_with_groups.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in a Verily Workbench notebook instance
    </a>
  </td>                                                                                               
</table>

## Overview

Access to resources and workspaces in Verily Workbench is handled by [Verily Workbench groups](https://terra-docs.api.verily.com/docs/how_to_guides/using_groups/). This notebook provides a series of widgets that enable you to perform various tasks related to groups in Verily Workbench. All of the tasks accomplished by widgets in this notebook can also be accomplished via [Workbench CLI](https://terra-docs.api.verily.com/docs/getting_started/install_and_run/) commands in your cloud environment's termnal. 

### Objective

Perform common workspace resource operations including:

- [View the Verily Workbench groups in which you're a member](#view_groups)
- [Create a new Verily Workbench group](#create_group)
- [Delete an existing Verily Workbench group](#delete_group)
- [Add and remove collaborators to and from your Verily Workbench group](#manage_membership)

#### How to run this notebook

Please run the Setup section before running any other sections in this worksapce. 

#### Costs

This notebook takes less than a minute to run, which will typically cost less than $0.01 of compute time on your cloud environment.

## Setup

Run the cell below to set up utilities for the widgets provided in this workspace.

In [ ]:
# Widget utilities
from IPython.display import display, HTML
import csv
import json
import ipywidgets as widgets
import subprocess

def users_table(stdout):
    """ Create table from JSON string
    """
    html = f"""<table style='margin: 0 auto,text-align: left'>"""
    json_data = json.loads(stdout)
    html += "<th>NAME</th>"
    html += "<th>EMAIL</th>"
    html += "<th>MEMBERS</th>"
    html += "<th>POLICIES</th>"
    for row in json_data:
        html += "<tr>"
        html += f"<td>{row['name']}</td>"
        html += f"<td>{row['email']}</td>"
        html += f"<td>{row['numMembers']}</td>"
        html += f"<td>{row['currentUserPolicies'][0]}</td>"
    html += "</table>"
    return html


input_style = {
    'description_width':'initial'
}

input_layout = widgets.Layout(
    width='50%',
    align_items='flex-start'
)

output_style = {
    'description_width':'initial',
}

class TextInputWidget(object):
    def __init__(self, ph, desc):
        self.description = desc
        self.placeholder = ph
        self.widget = widgets.Text(
            placeholder=self.placeholder,
            description=self.description,
            layout=input_layout,
            style=input_style)

    def get(self):
        return self.widget


class DropdownInputWidget(object):
    def __init__(self, options, value, description):
        self.options = options
        self.value = value
        self.description = description
        self.widget = widgets.Dropdown(
            options=self.options,
            value=self.value,
            disabled=False,
            description=self.description,
            layout=input_layout,
            style=input_style
        )

    def get(self):
        return self.widget
    
class StyledButton(object):
    def __init__(self,desc,tooltip,icon):
        self.description = desc
        self.tooltip = tooltip
        self.icon = icon
        self.button = widgets.Button(
                description=self.description,
                disabled=False,
                layout = widgets.Layout(width='50%'),
                display='flex',
                align_items='stretch',
                button_style='',
                tooltip=self.tooltip,
                icon=self.icon,
                style=widgets.ButtonStyle(button_color = '#D8D2EB')
        )
    def get(self):
        return self.button

## View your Verily Workbench group memberships
<a id='view_groups'></a>

Run the cell below to view a list of the Verily Workbench groups in which you are a member. You should be a member of at least one group, specific to your organization, which will have the name `<ORGANIZATION>-users`.

In [ ]:
result = subprocess.run(['terra','group','list','--format=JSON'],capture_output=True,text=True)
print(result.stderr) if not result.stdout else display(HTML(users_table(result.stdout)))

## View members of your Verily Workbench group

In order to view the members of a Verily Workbench group, you must be an owner of that group. Run the command below to view a list of the Verily Workbench groups in which you are a member. You should be a member of at least one group, specific to your organization, which will have the name `<ORGANIZATION>-users`.

To investigate the membership of your group, run the cell below to create a widget. Provide the group name in the widget input field and click the button to view a list of the members of the desired group.

In [ ]:
class ListMembersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.output = widgets.Output()
        self.list_button = StyledButton('List users in group','Click to list users in this Verily Workbench group.','list').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.list_button,self.output])
        self.list_button.on_click(self.list_members)

        
    def list_members(self,b):
        with self.output:
            terra_command = ["terra","group","list-users",f"--name={self.input_group_name.value}","--format=text"]
            result = subprocess.run(terra_command,capture_output=True,text=True)
            if not result.stdout:
                self.output.append_display_data(result.stderr)
            else:
                self.output.append_display_data(HTML(users_table(result.stdout)))

# Create widget                
list_members_widget = ListMembersWidget()
display(list_members_widget.vb)

## Create a new Verily Workbench group
<a id='create_group'></a>

Run the cell below to create a widget, then populate the widget's fields and click the button to create a new Verily Workbench group. 
The value provided for the `GROUP_NAME` should be unique, and reflect the purpose of the group (for example, for a group of Verily researchers working on the 1000 Genomes dataset, `verily-1000-genomes-researchers` would be appropriate).

You should see output resembling:

```
Terra group created.
<GROUP_NAME>
  Email: <GROUP_NAME>@verily.com
  # Members: 1
  Current user's policies: ADMIN
```

In [ ]:
class CreateGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.output = widgets.Output()
        self.button = StyledButton('Create group','Click to create a Verily Workbench group.','plus').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.button,self.output])
        self.button.on_click(self.create_group)

    def create_group(self,b):
        with self.output:
            self.output.clear_output()
            terra_command = ["terra","group","create",f"--name={self.input_group_name.value}","--format=text"]
            result = subprocess.run(terra_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
create_group_widget = CreateGroupWidget()
display(create_group_widget.vb)

## Delete a Verily Workbench group

<a id='delete_group'></a>

In order to delete a Verily Workbench group, you must be an admin of that group. Run the cell below to create a widget, then populate the input fields and click the button to delete the desired group.

**NOTE:** Once a Verily Workbench group has been deleted, this operation cannot be reversed. Please take care not to delete any groups currently in use.

Your output should resemble:
```
<GROUP_NAME>
  Email: <GROUP_NAME>@verily.com
  # Members: <NUMBER_OF_MEMBERS>
  Current user's policies: ADMIN
Terra group successfully deleted.
```

In [ ]:
class DeleteGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.output = widgets.Output()
        self.button = StyledButton('Delete group','Click to delete a Verily Workbench group.','trash').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.button,self.output])
        self.button.on_click(self.delete_group)

    def delete_group(self,b):
        with self.output:
            self.output.clear_output()
            terra_command = ["terra","group","delete",f"--name={self.input_group_name.value}","--quiet"]
            result = subprocess.run(terra_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
delete_group_widget = DeleteGroupWidget()
display(delete_group_widget.vb)

## Manage Verily Workbench group membership
<a id='manage_membership'></a>

Managing group membership is often an ongoing task throught a project's lifecycle.
<br>The sub-sections below create widgets that empower you to manage group membership without having to use the Workbench CLI in the terminal.

**Note:** In order to manage the membership of a Verily Workbench group, you must have the 'ADMIN' policy in that group.

### Add a user to a Verily Workbench group

1. Run the cell below to create a widget that adds a user to a Verily Workbench group.
1. In the widget, provide the group name for an existing Verily Workbench group and the email of the user you wish to add. 
1. Click the button to add the user to the group!

You should then see output resembling:
```
User added to Terra group.
<EMAIL>: <POLICY>
```

In [ ]:
class AddUserWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_user_email = TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.policy_drop_down = DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Policy:").get()
        self.output = widgets.Output()
        self.button = StyledButton('Add user','Click to add a user to the group.','user-plus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_user_email,
             self.policy_drop_down,
             self.button,
             self.output
            ],style=input_style,layout=widgets.Layout(align_items='flex-start'))
        self.button.on_click(self.add_user)

    def add_user(self,b):
        with self.output:
            # self.output.clear_output()
            terra_command = ["terra", "group", "add-user",f"--name={self.input_group_name.value}",f"--email={self.input_user_email.value}",f"--policy={self.policy_drop_down.value}"]
            result = subprocess.run(terra_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
add_user_widget = AddUserWidget()
display(add_user_widget.vb)

### Remove a user from a Verily Workbench group

1. Run the cell below to create a widget.
1. In the widget, provide the group name for an existing Verily Workbench group and the email of the user you wish to remove. 
1. Click the button to remove the user from the group.

You should see output resembling:
```
User (<USER_EMAIL>) removed from policy (<POLICY>) in group (<GROUP_NAME>).
```

In [ ]:
class RemoveUserWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_user_email = TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.policy_drop_down = DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Policy:").get()
        self.output = widgets.Output()
        self.button = StyledButton('Remove user','Click to remove a user from the group.','user-minus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_user_email,
             self.policy_drop_down,
             self.button,
             self.output
            ],style=input_style,layout=widgets.Layout(align_items='flex-start'))
        self.button.on_click(self.remove_user)

    def remove_user(self,b):
        with self.output:
            # self.output.clear_output()
            terra_command = ["terra", "group", "remove-user",f"--name={self.input_group_name.value}",f"--email={self.input_user_email.value}",f"--policy={self.policy_drop_down.value}"]
            result = subprocess.run(terra_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
add_user_widget = RemoveUserWidget()
display(add_user_widget.vb)

### Add a batch of users to a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "TERRA_USER_EMAIL" and "POLICY", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding membership policy (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to add a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User added to Terra group.
<EMAIL>: <POLICY>
```

In [ ]:
class BatchAddUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:",).get()
        self.input_file = TextInputWidget('<FILE>',"CSV File:",).get()
        self.output = widgets.Output()
        self.button = StyledButton('Add users','Click to add a batch of users to the group.','user-plus').get()
        self.button.on_click(self.batch_add_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_file,self.button,self.output],
                          style=input_style,layout=widgets.Layout(align_items='flex-start'))

    def batch_add_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    terra_command = ["terra", "group", "add-user",f"--name={self.input_group_name.value}",f"--email={user['TERRA_USER_EMAIL']}",f"--policy={user['POLICY']}"]
                    result = subprocess.run(terra_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)



# Instantiate widget
batch_add_users_widget = BatchAddUsersWidget()
display(batch_add_users_widget.vb)

### Remove a batch of users from a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "TERRA_USER_EMAIL" and "POLICY", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding membership policy (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to remove a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User (<USER_EMAIL>) removed from policy (<POLICY>) in group (<GROUP_NAME>).
```

In [ ]:
class BatchRemoveUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_file = TextInputWidget('<FILE>',"CSV File:").get()
        self.output = widgets.Output()
        self.button = StyledButton('Remove users','Click to remove a batch of users to the group.','user-minus').get()
        self.button.on_click(self.batch_remove_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_file,self.button,self.output],
                          style=input_style,layout=widgets.Layout(align_items='flex-start'))

    def batch_remove_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    terra_command = ["terra", "group", "remove-user",f"--name={self.input_group_name.value}",f"--email={user['TERRA_USER_EMAIL']}",f"--policy={user['POLICY']}"]
                    result = subprocess.run(terra_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)



# Instantiate widget
batch_remove_users_widget = BatchRemoveUsersWidget()
display(batch_remove_users_widget.vb)

## Provenance

Generate information about this notebook environment and the packages installed.

In [ ]:
!date

Conda and pip installed packages:

In [ ]:
!conda env export

JupyterLab extensions:

In [ ]:
!jupyter labextension list

Number of cores:

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

Memory:

In [ ]:
!grep "^MemTotal:" /proc/meminfo

---
Copyright 2022 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd